In [ ]:
import os

from database import data_utils

import contextily as cx
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from shapely.ops import nearest_points

In [ ]:
# Get traces from all data that was collected during a validation day
data = data_utils.combine_all_folder_data("../data/kcm_validation")
traces = data_utils.calculate_trace_df(data, 'file', 'tripid', 'locationtime', 'lat', 'lon', ['orientation','scheduledeviation','tripdistance','locationtime'], use_coord_dist=True)
# Add geometry
traces = geopandas.GeoDataFrame(traces, geometry=geopandas.points_from_xy(traces.lon, traces.lat), crs="EPSG:4326")

# # Get traces from all data that was collected during a validation day
# data = data_utils.combine_all_folder_data("../data/nwy_all", n_sample=10)
# traces = data_utils.calculate_trace_df(data, 'file', 'datedvehiclejourney', 'locationtime', 'lat', 'lon', ['bearing','locationtime'], use_coord_dist=True)
# # Add geometry
# traces = geopandas.GeoDataFrame(traces, geometry=geopandas.points_from_xy(traces.lon, traces.lat), crs="EPSG:4326")

traces

In [ ]:
# Map every trace to a network segment
with open('../data/processed_networks/kcm_segments.pkl', 'rb') as f:
    kcm_segments = pickle.load(f)

# Convert to WA north projection
kcm_segments = kcm_segments.to_crs('EPSG:32148')
traces = traces.to_crs('EPSG:32148')

kcm_segments

In [ ]:
# Sample from the traces, match to the network
traces_n = traces.sample(100000)
# There are duplicates when distance to segments is tied; just take first
traces_n = geopandas.sjoin_nearest(traces_n, kcm_segments, distance_col="join_dist").drop_duplicates(['tripid','locationtime'])
traces_n

In [ ]:
traces.columns

In [ ]:
traces.groupby(['vehicle']).agg(['count','mean','nunique'])